In [1]:
import pandas as pd
import numpy as np
import pickle
import glob
import os
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import cross_val_score
#Supress default INFO logging
import logging
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)
plt.rcParams['font.family'] = 'NanumGothic'
plt.rcParams['font.family'] = 'Malgun Gothic'


# data load  and preprocessing

In [2]:
defende_path = "./data/수비/"
batter_path = "./data/타자/"
file_list = ['2011.tsv','2012.tsv','2013.tsv','2014.tsv','2015.tsv','2016.tsv','2017.tsv','2018.tsv','2019.tsv']

In [3]:
def data_load(file_list, defende_path,batter_path):
    df = pd.DataFrame()
    for data_path in file_list:
        defender = pd.read_csv(defende_path + data_path,sep="\t")
        batter = pd.read_csv(batter_path + data_path,sep="\t")
        data = pd.merge(defender,batter,on=["선수명","팀명"])
        df = pd.concat([df,data])
    return df, list(defender.columns)

In [4]:
def del_noise_column(data, columns):
    noise_columns = ["순위_x","순위_y","G_x"] + columns
    for column in noise_columns:
        if column in data:
            del data[column]
    return data

In [5]:
#### defender_coumns 제거를 위해  수비수 포지션을 받고  포지션은 제거 리스트에서 제외

In [6]:
df, defender_coumns = data_load(file_list, defende_path, batter_path)
del defender_coumns[defender_coumns.index("POS")]
del defender_coumns[defender_coumns.index("선수명")]
del defender_coumns[defender_coumns.index("팀명")]

In [7]:
df = del_noise_column(df,list(defender_coumns))

In [8]:
if "선수명" in df:
    df_name = df["선수명"]
    del df["선수명"]

label = None
if "POS" in df:
    label = df["POS"]
    del df["POS"]

In [9]:
del df["팀명"]
# 팀명 원핫 인코딩
# if "팀명" in df:
#     one_hot_encoded = pd.get_dummies(df.팀명) 
#     del df["팀명"]
#     df = pd.concat([one_hot_encoded,df],axis=1)

# 테스트 추가 데이터 로드

In [16]:
test_path = "./data/test/2019.tsv"
game_data_path = "./data/컴투스프로야구_수비/02_08.tsv"

In [19]:
test_data = pd.read_csv(test_path,sep="\t")
game_data = pd.read_csv(game_data_path,sep="\t")

In [24]:
pd.merge(test_data,game_data, left_on="선수명",right_on="선수").to_csv("test.csv")
